In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tsai.all as ts
import torch
from torch import nn
from my_utilities import time_frequency_analysis, mma_Partition, time_frequency_analysis_output_1s
from sklearn.linear_model import RidgeClassifierCV

import time

In [15]:
srate = 2000
new_srate = 200
sec = 3 # time step length
ele = pd.read_csv('data/ele_4_60_hz.csv', header = None, index_col = None).to_numpy()
# shape = (4, 15, 2, 20000, 14)
# 去除第一秒的数据,且重采样到200hz
ele = np.reshape(
    ele, (20000,15,4,2,14), order = 'F').transpose([2,1,3,0,4])[:,:,:,srate::srate//new_srate,:]

# 分成头部与耳部，标注label
ear_ele = ele[...,:8].astype(np.float32).reshape(-1,int(200 * sec),8)
head_ele = ele[...,8:].astype(np.float32).reshape(-1,int(200 * sec),6)
# labels = np.repeat([0,1,2,3], ear_ele.shape[0]//4)

In [16]:
# batches, channels, time step
ear_ele = ele[...,:8].astype(np.float32).reshape(-1,int(200 * sec),8).transpose([0,2,1])
head_ele = ele[...,8:].astype(np.float32).reshape(-1,int(200 * sec),6).transpose([0,2,1])

# z-score
head_ele = (head_ele - head_ele.mean(axis = 2)[...,np.newaxis])\
                /head_ele.std(axis = 2)[...,np.newaxis]

ear_ele = (ear_ele - ear_ele.mean(axis = 2)[...,np.newaxis])\
                /ear_ele.std(axis = 2)[...,np.newaxis]

y = np.repeat([0,1,2,3], int(ear_ele.shape[0]/4))

# ################################ TFA #########################
# mask = np.unique(np.array(list(range(5,59,5))+
#                           list(range(7,59,7))+
#                           list(range(9,59,9))+
#                           list(range(11,59,11))))-2



# ear_X_train, ear_X_test, ear_y_train, ear_y_test = train_test_split(ear_ele, y, test_size=0.1)
# # head_X_train, head_X_test, head_y_train, head_y_test = train_test_split(head_ele, y, test_size=0.1)
# # ################# DATA generate ##############################
# # ear_X_train, ear_y_train = roll_window(ear_X_train, ear_y_train)
# # ear_X_train, ear_y_train = shuffle(ear_X_train, ear_y_train)
# ##############################################################
# ear_ele, ear_y, ear_splits = ts.combine_split_data([ear_X_train, ear_X_test], [ear_y_train, ear_y_test])

In [17]:
ear_ele.shape, y.shape

((360, 8, 600), (360,))

In [18]:
def cross_validation(x, y, cv_number = 20, test_size = 0.1):
    acc = []
    batch_tfms = [ts.TSStandardize(by_sample=True)]
    tfms  = [None, [ts.Categorize()]]
    
    for _ in np.arange(cv_number):
        x, y = shuffle(x, y)
        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=test_size)
        
            
        ear_X, ear_y, splits = ts.combine_split_data([X_train, X_test], [y_train, y_test])
        dls = ts.get_ts_dls(ear_X, ear_y, 
                         splits=splits, 
                         tfms=tfms, drop_last=False, 
                         shuffle_train=False, batch_tfms=batch_tfms, bs=10_000)        
        model = ts.build_ts_model(ts.ROCKET, dls=dls)
        
        X_train, y_train = ts.create_rocket_features(dls.train, model)
        X_valid, y_valid = ts.create_rocket_features(dls.valid, model)
        X_train.shape, X_valid.shape
    
        ridge = RidgeClassifierCV(alphas=np.logspace(-8, 8, 17), normalize=True)
        ridge.fit(X_train, y_train)
        print(f'alpha: {ridge.alpha_:.2E}  train: {ridge.score(X_train, y_train):.5f}  valid: {ridge.score(X_valid, y_valid):.5f}')
        acc.append(ridge.score(X_valid, y_valid))
        
    return acc

In [19]:
acc = cross_validation(ear_ele, y, cv_number = 20, test_size = 0.1)

alpha: 1.00E+08  train: 0.26852  valid: 0.08333
alpha: 1.00E+08  train: 0.26543  valid: 0.11111
alpha: 1.00E+08  train: 0.25617  valid: 0.19444
alpha: 1.00E+08  train: 0.26235  valid: 0.13889
alpha: 1.00E+08  train: 0.25926  valid: 0.16667
alpha: 1.00E+08  train: 0.52160  valid: 0.08333
alpha: 1.00E+08  train: 0.26235  valid: 0.13889
alpha: 1.00E+08  train: 0.25617  valid: 0.19444
alpha: 1.00E+08  train: 0.25926  valid: 0.16667
alpha: 1.00E+08  train: 0.26235  valid: 0.13889
alpha: 1.00E+08  train: 0.51235  valid: 0.11111
alpha: 1.00E+08  train: 0.25617  valid: 0.19444
alpha: 1.00E+08  train: 0.25617  valid: 0.19444
alpha: 1.00E+08  train: 0.26543  valid: 0.11111
alpha: 1.00E+08  train: 0.26235  valid: 0.13889
alpha: 1.00E+08  train: 0.26235  valid: 0.13889
alpha: 1.00E+08  train: 0.26543  valid: 0.11111
alpha: 1.00E+08  train: 0.25926  valid: 0.16667
alpha: 1.00E+08  train: 0.26235  valid: 0.13889
alpha: 1.00E+08  train: 0.25617  valid: 0.19444


In [14]:
# pd.DataFrame(acc, columns=['4s']).to_csv(r'result/ORIGINAL_ROCKET_ACC.csv', index= None)